In [194]:
from itertools import product
from random import random, randint, shuffle, seed, choice, getrandbits
import numpy as np
from scipy import sparse
from collections import namedtuple
from functools import reduce
from copy import copy
import time

In [195]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets


# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

- `num_points = [100, 1_000, 5_000]`
- `num_sets = num_points `
- `density = [.3, .7] `


*Problem: * Number of evaluations / Best fitness



In [196]:
x = make_set_covering_problem(1000, 1000, .3)
print("Element at row=42 and column=42:", x[42, 42])

Element at row=42 and column=42: False


In [197]:
NUM_POINTS = 10
NUM_SETS = 10
density = .2

sets = make_set_covering_problem(NUM_POINTS, NUM_SETS, density)
State = namedtuple('State', ['taken', 'not_taken'])
#utils functions
def is_solvable(problem_instance):
    return np.all(problem_instance.sum(axis=0))

"""
def using_nonzero_iteration(x):
    rows,cols = x.nonzero()
    for row,col in zip(rows,cols):
        print((row,col), x[row,col])
"""

def goal_check(sets, state):
    covered_points = [False] * (len(state.taken) + len(state.not_taken))
    rows,cols = sets.nonzero()
    for row,col in zip(rows,cols):
        #print((row,col), sets[row,col])
        if row in state.taken:
            covered_points[col] = True
    return np.all(covered_points)
    

print(is_solvable(sets))
assert(goal_check(sets, State(set(range(NUM_SETS)), set())), "Problem not solvable")

True


<>:29: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:29: SyntaxWarning: assertion is always true, perhaps remove parentheses?
C:\Users\Lorenzo\AppData\Local\Temp\ipykernel_16112\2816762034.py:29: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(goal_check(sets, State(set(range(NUM_SETS)), set())), "Problem not solvable")


In [198]:
NUM_POINTS = 100
NUM_SETS = 100
density = .7

sets = make_set_covering_problem(NUM_POINTS, NUM_SETS, density)
State = namedtuple('State', ['taken', 'not_taken'])

def quality(state):
    #this counts, right now, the number of covered points
    covered_points = [False] * NUM_POINTS
    rows,cols = sets.nonzero()
    for row,col in zip(rows,cols):
        #print((row,col), sets[row,col])
        if row in state.taken:
            covered_points[col] = True
    return np.array(covered_points).sum()

def tweak(state): # In case of ES I should use a Normal/guassian distribution for introducing variability. Larger sigma leads to more exploration.
    #this function triggers a small random change in the current state
    # IN: state
    # OUT: tweaked(state) with random change
    seed(time.time())
    c = choice(list(state.not_taken))
    return State(
                state.taken ^ {c},
                state.not_taken ^ {c},
            )

state = State(set(), set(range(NUM_SETS)))
assert(quality(state)) == 0
tweaked = tweak(state)
print(tweaked)

State(taken={96}, not_taken={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 97, 98, 99})


In [199]:
EVALUATIONS = 2*NUM_SETS

def generate_state():
    taken = set()
    not_taken = set(range(NUM_SETS))
    """
    #Commented because generates an intermediate state, here I want to solve the problem from scratch
    for i in range(0, NUM_SETS):
        if bool(getrandbits(1)):
            taken = taken ^ {i}
            not_taken = not_taken ^ {i}
    """
    return State(taken, not_taken)

def basic_hillclimbing(SETS):
    i = 0
    # Generate an intermediate state to start from
    current_solution = generate_state()
    print(f"STARTING HILL CLIMBING: {current_solution} | quality={quality(current_solution)}")         
    while i < EVALUATIONS and not goal_check(SETS, current_solution): #SHOULD BE until S is the ideal solution or we have run out of time
        R = tweak(copy(current_solution))
        if quality(R) > quality(current_solution):
            #print(f"{i}) Quality(old)={quality(current_solution)} WHERE Quality(new)={quality(R)} , {R}")
            current_solution = R
        i = i+1
    print(f"\n[STEPS={i}] Current solution: {current_solution} | quality={quality(current_solution)} | solution_size={len(current_solution.taken)}")
    return current_solution

In [200]:
basic_hillclimbing(sets)

STARTING HILL CLIMBING: State(taken=set(), not_taken={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99}) | quality=0

[STEPS=3] Current solution: State(taken={92, 14, 87}, not_taken={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 88, 89, 90, 91, 93, 94, 95, 96, 97, 98, 99}) | quality=100 | solution_size=3


State(taken={92, 14, 87}, not_taken={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 88, 89, 90, 91, 93, 94, 95, 96, 97, 98, 99})

In [201]:
def second_hillclimbing(SETS):
    i = 0
    n = NUM_POINTS
    # Generate an intermediate state to start from
    current_solution = generate_state()
    print(f"STARTING HILL CLIMBING: {current_solution} | quality={quality(current_solution)}")         
    while i < EVALUATIONS and not goal_check(SETS, current_solution): #SHOULD BE until S is the ideal solution or we have run out of time
        R = tweak(copy(current_solution))
        for j in range(n-1):
            W = tweak(copy(current_solution))
            if quality(W) > quality(R):
                R = W
        if quality(R) > quality(current_solution):
            current_solution = R
            #print(f"{i}) Quality(old)={quality(current_solution)} WHERE Quality(new)={quality(R)} , {R}")
        i = i+1
    print(f"\n[STEPS={i}] Current solution: {current_solution} | quality={quality(current_solution)} | solution_size={len(current_solution.taken)}")
    return current_solution

In [202]:
second_hillclimbing(sets)

STARTING HILL CLIMBING: State(taken=set(), not_taken={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99}) | quality=0

[STEPS=3] Current solution: State(taken={96, 65, 58}, not_taken={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 59, 60, 61, 62, 63, 64, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 97, 98, 99}) | quality=100 | solution_size=3


State(taken={96, 65, 58}, not_taken={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 59, 60, 61, 62, 63, 64, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 97, 98, 99})

In [203]:
PROBLEM_SIZE = 100
NUM_SETS = 100
SETS = make_set_covering_problem(PROBLEM_SIZE, NUM_SETS, density)
State = namedtuple('State', ['taken', 'not_taken'])

def fitness1(state,x):
    cost = sum(state)
    valid = np.all(
        reduce(
            np.logical_or,
            [x.getrow(i).toarray() for i, t in enumerate(state) if t],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )
    return valid, -cost

def fitness2(state,x):
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [x.getrow(i).toarray() for i, t in enumerate(state) if t],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )
    return valid, -cost
    
fitness = fitness2

In [204]:
def tweak(state):
    new_state = copy(state)
    index = randint(0, PROBLEM_SIZE - 1)
    new_state[index] = not new_state[index]
    return new_state

In [205]:
#Traditional Hill climbing
current_state = [choice([False, False, False, False, False, False]) for _ in range(NUM_SETS)]
print(fitness(current_state, SETS))

for step in range(10_000):
    new_state = tweak(current_state)
    if fitness2(new_state, SETS) >= fitness2(current_state, SETS):
        current_state = new_state
        #print(f"Current state is: {current_state} and fitness={fitness(current_state)}")
        print(f"fitness={fitness(current_state, SETS)}")

(0, 0)
fitness=(71, -1)
fitness=(95, -2)
fitness=(98, -3)
fitness=(100, -4)


In [207]:
#Tabu search algorithm
SIZE = 100
sets = make_set_covering_problem(NUM_POINTS, NUM_SETS, density)
current_state = [choice([False, True]) for _ in range(NUM_SETS)]

def goal_check(state, s_sets):
    return np.all(
        reduce(
            np.logical_or,
            [s_sets.getrow(i).toarray() for i, t in enumerate(state) if t],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )


tabu_list = []
num_iterations = 100
best_solution = current_state
best_solution_fitness = fitness(best_solution, sets)
step = 0

for i in range(num_iterations):
    #Candidate List of Moves
    candidates = [tweak(current_state) for _ in range(100)]
    #Select the best admissible candidate
    tmp_best = candidates[0]
    for ng in candidates:
        if ng not in tabu_history and fitness(ng, sets) >= fitness(tmp_best, sets):
            tmp_best = ng
            step += 1

    #Update Admissibility conditions
    tabu_list.append(tmp_best)
    #Update best solution ever
    if fitness(tmp_best, sets) > best_solution_fitness:
        best_solution = tmp_best
        best_solution_fitness = fitness(tmp_best, sets)
    
    while len(tabu_list) > SIZE:
        tabu_list.pop(0)

    print(f"Iterations done: {i}, step={step} best_solution=..., best_cost={fitness(best_solution, sets)}")
    print(f"goal_check={goal_check(best_solution, sets)}")
    

Iterations done: 0, step=45 best_solution=..., best_cost=(100, -43)
goal_check=True
Iterations done: 1, step=90 best_solution=..., best_cost=(100, -43)
goal_check=True
Iterations done: 2, step=133 best_solution=..., best_cost=(100, -43)
goal_check=True
Iterations done: 3, step=182 best_solution=..., best_cost=(100, -43)
goal_check=True
Iterations done: 4, step=233 best_solution=..., best_cost=(100, -43)
goal_check=True
Iterations done: 5, step=277 best_solution=..., best_cost=(100, -43)
goal_check=True
Iterations done: 6, step=327 best_solution=..., best_cost=(100, -43)
goal_check=True
Iterations done: 7, step=369 best_solution=..., best_cost=(100, -43)
goal_check=True
Iterations done: 8, step=419 best_solution=..., best_cost=(100, -43)
goal_check=True
Iterations done: 9, step=460 best_solution=..., best_cost=(100, -43)
goal_check=True
Iterations done: 10, step=507 best_solution=..., best_cost=(100, -43)
goal_check=True
Iterations done: 11, step=550 best_solution=..., best_cost=(100, -